In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm
import pandas as pd
from data_sources import process_parallel, transform_csv_row

In [4]:
from collections import Counter

## Encode music21 stream to text representation 

This notebook uses a full component format 
- measure separators, instruments, and separated octaves
- Format: note, octave, action type, instrument
- note repr: nG# o4 t1 i0

### Load midi data

In [5]:
version = 'v8'
data_path = Path('data/midi')
version_path = data_path/version

In [6]:
source_dir = 'midi_npz'

out_dir = 'midi_encode/np/shortdur_2c_12range'
num_comps = 2 # 2:(note,dur), 3:(note,dur,oct), 4:(note,dur,oct,inst)
source_csv = version_path/source_dir/f'{source_dir}.csv'
out_csv = version_path/out_dir/f'midi_encode.csv'

In [7]:
df = pd.read_csv(source_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,4,15,18,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,inferred_offset,song_url,instruments,ht_mode,midi_title,title,seconds,midi,inferred_key,quarter_length,...,ht_key,ht_time_signature,bpm,section,parts,genres,mxl,midi_transform,midi_npz,midi_npz_timesteps
0,0.0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,1.0,yu-gi-oh,yu-gi-oh-theme-song,25.411765,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,36.0,...,C,3.0,85.0,intro,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,145.0
1,0.0,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",1.0,yu-gi-oh3,yu-gi-oh-theme-song,15.000000,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,32.0,...,C,4.0,128.0,chorus,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,129.0
2,5.0,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",1.0,kiefer,kiefer,10.000000,midi_sources/hooktheory/pianoroll/w/what-a-day...,E minor,16.0,...,C,4.0,96.0,chorus,chorus,Jazz,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...,midi_npz/hooktheory/pianoroll/w/what-a-day/kie...,65.0
3,NaN,https://www.hooktheory.com/theorytab/view/weez...,NaN,1.0,My New Song,beverly-hills,NaN,midi_sources/hooktheory/pianoroll/w/weezer/bev...,NaN,NaN,...,C,4.0,NaN,intro-and-verse,intro-and-verse,NaN,NaN,NaN,NaN,NaN
4,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,Weezer - Fall Together,fall-together-,10.322581,midi_sources/hooktheory/pianoroll/w/weezer/fal...,A minor,16.0,...,C,4.0,93.0,chorus,chorus,Rock,NaN,midi_transform/hooktheory/pianoroll/w/weezer/f...,midi_npz/hooktheory/pianoroll/w/weezer/fall-to...,65.0


In [8]:
df_filtered = df.loc[df[source_dir].notna()]; df_filtered.shape

(162688, 25)

In [9]:
df_filter = (df_filtered.source != 'hooktheory') | (df_filtered.ht_time_signature == 4.0)
df_filtered = df_filtered.loc[df_filter]; df_filtered.shape

(160965, 25)

In [10]:
df_filtered = df_filtered.loc[df_filtered[f'{source_dir}_timesteps'] < 2e4]; df_filtered.shape

(160946, 25)

In [11]:
df_filtered[source_dir].values

array(['midi_npz/hooktheory/pianoroll/w/wayne-sharpe/yu-gi-oh-theme-song/chorus_key_cmajor.npz',
       'midi_npz/hooktheory/pianoroll/w/what-a-day/kiefer/chorus_key_cmajor.npz',
       'midi_npz/hooktheory/pianoroll/w/weezer/fall-together-/chorus_key_cmajor.npz',
       'midi_npz/hooktheory/pianoroll/w/wavves/afraid-of-heights/intro_key_cmajor.npz', ...,
       'midi_npz/130k_reddit/R/R/R.CONIFF.Medley.npz', 'midi_npz/130k_reddit/R/R/rubbersoul.npz',
       'midi_npz/130k_reddit/2/24preludescl.npz', 'midi_npz/130k_reddit/R/R/Rock-Around-Medley-(Medley).npz'],
      dtype=object)

In [21]:
def transform_func(file, out_file, row):
    chordarr = load_chordarr(file)
    seq = chordarr2seq(chordarr)
    npenc = seq2npenc(seq, num_comps=num_comps)
    if (npenc[...,1] > 128+ENC_OFFSET).any(): 
        print('npenc exceeds max 128 duration:', file)
        return None
    
    if ((npenc[...,0] > ENC_OFFSET) & ((npenc[...,0] < 12+ENC_OFFSET) | (npenc[...,0] >= 127-12+ENC_OFFSET))).any(): 
        print('npenc out of note range 12 - 116:', file)
        return None
    
    np.save(out_file, npenc)
    return npenc

In [22]:
# fp = Path('data/midi/v8')/df_filtered[source_dir].values[0]
# transform_func(fp, None, None).shape

In [23]:
from functools import partial
parallel_func = partial(transform_csv_row, 
        transform_func=transform_func,
        base_path=version_path,
        source_dir=source_dir,
        out_dir=out_dir,
        out_extension='.npy'
       )

In [24]:
for r in df_filtered.iterrows():
    parallel_func(r)
    break

In [25]:
encoded_files = process_parallel(parallel_func, df_filtered.iterrows(), total=df_filtered.shape[0])

npenc out of note range 12 - 116: data/midi/v8/midi_npz/freemidi/genre-dance-eletric/Chemical Brothers - Loops Of Furry.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/freemidi/genre-dance-eletric/Daniel Bedingfield - If You Are Not The One.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/freemidi/genre-dance-eletric/Modern Talking - Cheri Cheri Lady.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/freemidi/genre-dance-eletric/Girls Aloud - Jump.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/freemidi/genre-dance-eletric/Yellow Magic Orchestra - Cosmic Surfin.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/freemidi/genre-dance-eletric/Yellow Magic Orchestra - Waterford.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/freemidi/genre-pop/Aha - Blue Sky.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/freemidi/genre-dance-eletric/Aqua - Cartoon Heroes.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz

npenc out of note range 12 - 116: data/midi/v8/midi_npz/lmd_clean/Alan Parsons/Time.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/lmd_clean/Sophie B. Hawkins/Right Beside You.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/lmd_clean/Elvis/Blue Hawaii.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/lmd_clean/Genesis/Firth of Fifth.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/lmd_clean/Genesis/Supper's Ready.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/lmd_clean/Chris Isaak/Blue Hotel.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/lmd_clean/Golden Earring/Radar Love.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/lmd_clean/Korn/Porno Creep.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/lmd_clean/Jean Michel Jarre/Rendez-vous 2.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/lmd_clean/Jean Michel Jarre/Second Rendez-vous, Part 2.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/

npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/B/B/BECK.Devils haircut.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/B/B/Beck_-_Devil's_Haircut.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/B/B/Beck_Devils_Haircut.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/B/B/Babyface - The Day.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/B/B/baianidade.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/B/B/Beck - Devil's Haircut.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/B/B/BEEGEES.More than a woman K.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/L/L/lock_n_load__blow_ya_mind__chris_hodder.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/L/L/lochnacailli.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/L/L/Love-Song-1.npz
npenc out of note range 12 - 116: d

npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/Metal_Rock_rock.freemidis.net_MIDIRip/midi/k/korn/Porno_Creep.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/Metal_Rock_rock.freemidis.net_MIDIRip/midi/a/acdc/Rock_N_Roll_Aint_Noise_Pollution.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/Metal_Rock_rock.freemidis.net_MIDIRip/midi/a/acdc/Riff_Raff.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/Metal_Rock_rock.freemidis.net_MIDIRip/midi/a/acdc/Whole_Lotta_Rosie.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/Metal_Rock_rock.freemidis.net_MIDIRip/midi/y/yes/Yours_Is_No_Disgrace.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/Classical Archives - The Greats (MIDI)/Classical Piano Midis/Varios - Título desconocido/a_h/barcarol.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/Classical Archives - The Greats (MIDI)/Classical Piano Midis/Var

npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/Y/Y/yellow_magic_orchestra-waterford.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/G/G/grease06.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/G/G/Gecmiyor_gunler.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/G/G/girls_aloud-life_got_cold.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/G/G/GOLDEAGL.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/G/G/Genesis_-_Fifth_of_Fifth.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/G/G/GENESISSUPPERSREADY.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/G/G/G.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/G/G/gwen_stefani-hollaback_girl.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/G/G/gnaern.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/Classic

npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/R/R/Robert Gitelman - Children Of The Sun.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/R/R/RendezVous2.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/R/R/rammstein-engel.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/R/R/Radio Head - Karma Police II L.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/R/R/RendezVous2-3.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/R/R/RendezVous2-2.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/R/R/relax03.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/R/R/ramblingrose.npz
npenc exceeds max 128 duration: data/midi/v8/midi_npz/130k_reddit/R/R/Radar-Love-2.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/R/R/Rammstein_-_Engel.npz
npenc out of note range 12 - 116: data/midi/v8/midi_npz/130k_reddit/R/R/rua

In [36]:
tdf = pd.DataFrame(data={out_dir: list(encoded_files.values())}, index=list(encoded_files.keys()))

In [37]:
merged_df = df.join(tdf, how='outer'); tdf.shape, df.shape, merged_df.shape

((160946, 1), (185846, 25), (185846, 26))

In [38]:
merged_df.head()

,inferred_offset,song_url,instruments,ht_mode,midi_title,title,seconds,midi,inferred_key,quarter_length,...,ht_time_signature,bpm,section,parts,genres,mxl,midi_transform,midi_npz,midi_npz_timesteps,midi_encode/np/shortdur_2c_12range
0,0.0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,1.0,yu-gi-oh,yu-gi-oh-theme-song,25.411765,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,36.0,...,3.0,85.0,intro,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,145.0,NaN
1,0.0,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",1.0,yu-gi-oh3,yu-gi-oh-theme-song,15.000000,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,32.0,...,4.0,128.0,chorus,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,129.0,midi_encode/np/shortdur_2c_12range/hooktheory/...
2,5.0,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",1.0,kiefer,kiefer,10.000000,midi_sources/hooktheory/pianoroll/w/what-a-day...,E minor,16.0,...,4.0,96.0,chorus,chorus,Jazz,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...,midi_npz/hooktheory/pianoroll/w/what-a-day/kie...,65.0,midi_encode/np/shortdur_2c_12range/hooktheory/...
3,NaN,https://www.hooktheory.com/theorytab/view/weez...,NaN,1.0,My New Song,beverly-hills,NaN,midi_sources/hooktheory/pianoroll/w/weezer/bev...,NaN,NaN,...,4.0,NaN,intro-and-verse,intro-and-verse,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,Weezer - Fall Together,fall-together-,10.322581,midi_sources/hooktheory/pianoroll/w/weezer/fal...,A minor,16.0,...,4.0,93.0,chorus,chorus,Rock,NaN,midi_transform/hooktheory/pianoroll/w/weezer/f...,midi_npz/hooktheory/pianoroll/w/weezer/fall-to...,65.0,midi_encode/np/shortdur_2c_12range/hooktheory/...


In [40]:
merged_df.to_csv(out_csv, index=False); merged_df.head()

,inferred_offset,song_url,instruments,ht_mode,midi_title,title,seconds,midi,inferred_key,quarter_length,...,ht_time_signature,bpm,section,parts,genres,mxl,midi_transform,midi_npz,midi_npz_timesteps,midi_encode/np/shortdur_2c_12range
0,0.0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,1.0,yu-gi-oh,yu-gi-oh-theme-song,25.411765,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,36.0,...,3.0,85.0,intro,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,145.0,NaN
1,0.0,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",1.0,yu-gi-oh3,yu-gi-oh-theme-song,15.000000,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,32.0,...,4.0,128.0,chorus,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,129.0,midi_encode/np/shortdur_2c_12range/hooktheory/...
2,5.0,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",1.0,kiefer,kiefer,10.000000,midi_sources/hooktheory/pianoroll/w/what-a-day...,E minor,16.0,...,4.0,96.0,chorus,chorus,Jazz,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...,midi_npz/hooktheory/pianoroll/w/what-a-day/kie...,65.0,midi_encode/np/shortdur_2c_12range/hooktheory/...
3,NaN,https://www.hooktheory.com/theorytab/view/weez...,NaN,1.0,My New Song,beverly-hills,NaN,midi_sources/hooktheory/pianoroll/w/weezer/bev...,NaN,NaN,...,4.0,NaN,intro-and-verse,intro-and-verse,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,Weezer - Fall Together,fall-together-,10.322581,midi_sources/hooktheory/pianoroll/w/weezer/fal...,A minor,16.0,...,4.0,93.0,chorus,chorus,Rock,NaN,midi_transform/hooktheory/pianoroll/w/weezer/f...,midi_npz/hooktheory/pianoroll/w/weezer/fall-to...,65.0,midi_encode/np/shortdur_2c_12range/hooktheory/...
